In [ ]:
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker.debugger import Rule
from sagemaker.debugger import rule_configs


In [3]:
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

sm_sess = sagemaker.session.Session()

AWS Region: eu-west-3
RoleArn: arn:aws:iam::260598086981:role/service-role/AmazonSageMaker-ExecutionRole-20210916T222927


In [5]:
bucket = sagemaker.Session().default_bucket()
bucket

'sagemaker-eu-west-3-260598086981'

In [6]:
prefix = "demo-sagemaker-xgboost-boston"

In [7]:
s3_output_location='s3://{}/{}/{}'.format(bucket, prefix, 'xgboost_model')
s3_output_location

's3://sagemaker-eu-west-3-260598086981/demo-sagemaker-xgboost-boston/xgboost_model'

In [8]:
container=sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
print(container)

659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-xgboost:1.2-1


In [16]:
train_input = TrainingInput(
    f"s3://{bucket}/{prefix}/data/train.csv", content_type="csv"
)
validation_input = TrainingInput(
    f"s3://{bucket}/{prefix}/data/validation.csv", content_type="csv"
)

___

In [9]:
xgb_model=sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=s3_output_location,
    sagemaker_session=sm_sess,

)

In [ ]:
xgb_model.set_hyperparameters(
    eta = 0.2,
    gamma = 4,
    max_depth = 5,
    min_child_weight = 6,
    subsample = 0.7,
    objective = "reg:squarederror",
    eval_metric='rmse', 
    num_round = 1000
)

In [ ]:
xgb_model.fit({"train": train_input, "validation": validation_input}, wait=True)

In [33]:
xgb_model.output_path

's3://sagemaker-eu-west-3-260598086981/demo-sagemaker-xgboost-boston/xgboost_model'

In [34]:
xgb_model.latest_training_job.name

'sagemaker-xgboost-2021-09-23-20-35-13-230'

In [35]:
rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive

2021-09-23 20:43:14     404370 demo-sagemaker-xgboost-boston/xgboost_model/sagemaker-xgboost-2021-09-23-20-35-13-230/rule-output/CreateXgboostReport/xgboost_report.html
2021-09-23 20:43:13     209368 demo-sagemaker-xgboost-boston/xgboost_model/sagemaker-xgboost-2021-09-23-20-35-13-230/rule-output/CreateXgboostReport/xgboost_report.ipynb
2021-09-23 20:39:26     322345 demo-sagemaker-xgboost-boston/xgboost_model/sagemaker-xgboost-2021-09-23-20-35-13-230/rule-output/ProfilerReport-1632429313/profiler-output/profiler-report.html
2021-09-23 20:39:25     168692 demo-sagemaker-xgboost-boston/xgboost_model/sagemaker-xgboost-2021-09-23-20-35-13-230/rule-output/ProfilerReport-1632429313/profiler-output/profiler-report.ipynb
2021-09-23 20:39:21        190 demo-sagemaker-xgboost-boston/xgboost_model/sagemaker-xgboost-2021-09-23-20-35-13-230/rule-output/ProfilerReport-1632429313/profiler-output/profiler-reports/BatchSize.json
2021-09-23 20:39:21        198 demo-sagemaker-xgboost-boston/xgboost_mode

In [38]:
xgb_model.model_data

's3://sagemaker-eu-west-3-260598086981/demo-sagemaker-xgboost-boston/xgboost_model/sagemaker-xgboost-2021-09-23-20-35-13-230/output/model.tar.gz'

___
# Deploy

In [40]:
from sagemaker.serializers import CSVSerializer

xgb_predictor=xgb_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    serializer=CSVSerializer()
)


------------------!